### Assignment- I Solution 
#### Python script to progatamically fetch trade data from Binance and store in pandas dataframe

In [1]:
#Importing required libraries
import datetime
import timeit
import pandas as pd
import traceback

In [2]:
#Helper functions

def get_dates(start_date=None,end_date=None,n_days=None):
    '''
    Function to create n sequential dates
    
    Params:
    
    start_date: start date in the format '%Y-%m-%d' in UTC timezone
    end_date:  end date in the format '%Y-%m-%d' in UTC timezone
    n_days:    number of recent days
    
    Mandatory Params: 
    
    (start_date and end_date) or n_days
    
    Precedence of Params:
    n_days has higher precedence over (start_date and end_date)
    '''    

    if n_days is None:
        if start_date is None or end_date is None:
            return "Download Failed: Mandatory params missing"
        else:    
            start_date_dt=datetime.datetime.strptime(start_date,'%Y-%m-%d')
            end_date_dt=datetime.datetime.strptime(end_date,'%Y-%m-%d')
            days=0
            dates=[]


            while (start_date_dt+datetime.timedelta(days=days))<=end_date_dt:
                dates.append((start_date_dt+datetime.timedelta(days=days)).strftime('%Y-%m-%d'))
                days+=1
            return dates
    else:
        utcnow=datetime.datetime.utcnow()
        utclastday=(utcnow-datetime.timedelta(days=1))
        days=0
        dates=[]
        while days<=n_days:
            dates.append((utclastday+datetime.timedelta(days=-1*days)).strftime('%Y-%m-%d'))
            days+=1
        return dates[::-1]
            

def download_data(days):
    '''
    Function to download trade data from binance open source end point for given list of days
    
    Params:
    days: List of days in format '%Y-%m-%d' in UTC timezone
    '''
    if type(days)!=list:
        return "Download Failed"
    print("Data Downloading for : " ,days)
    frames=[]
    utclastday=''
    for utclastday in days:
        try:
            url='https://data.binance.vision/data/futures/um/daily/trades/BTCUSDT/BTCUSDT-trades-{0}.zip'.format(utclastday)
            print(url)
            df=pd.read_csv(url,compression='zip',header=None)
            df['symbol']='BTCUSDT'
            df.columns=['id','price','qty','quote_qty','time','is_buyer_maker','symbol']
            df=df[['id','symbol','price','qty','quote_qty','time','is_buyer_maker']]
            df.set_index('id',inplace=True)
            frames.append(df)
        except Exception as ex:
            
            print(ex)
            print("Download Failed for ",utclastday)
    
    return "Couldn't download data for days" if len(frames)==0 else pd.concat(frames)      


In [3]:
#Downloader function
def data_downloader(start_date=None,end_date=None,n_days=None):
    '''
    Function to download trade data from binance open source end point
    
    Params:
    
    start_date: start date in the format '%Y-%m-%d' in UTC timezone from which the data hast to be downloaded
    end_date:  end date in the format '%Y-%m-%d' in UTC timezone to which the data hast to be downloaded
    n_days:    number of recent days for which the data has to be downloaded 
    
    Mandatory Params: 
    
    (start_date and end_date) or n_days
    
    Precedence of Params:
    n_days has higher precedence over (start_date and end_date)
    '''
    if n_days is None:
        if start_date is None or end_date is None:
            return "Download Failed: Mandatory params missing"
        else:
            return download_data(get_dates(start_date,end_date))
    else:
        return download_data(get_dates(n_days=n_days))


### Sample function call for the desired dates in the assessment


In [4]:
result=data_downloader(start_date='2022-03-01',end_date='2022-03-03')
print(result)


Data Downloading for :  ['2022-03-01', '2022-03-02', '2022-03-03']
https://data.binance.vision/data/futures/um/daily/trades/BTCUSDT/BTCUSDT-trades-2022-03-01.zip
https://data.binance.vision/data/futures/um/daily/trades/BTCUSDT/BTCUSDT-trades-2022-03-02.zip
https://data.binance.vision/data/futures/um/daily/trades/BTCUSDT/BTCUSDT-trades-2022-03-03.zip
             symbol    price    qty  quote_qty           time  is_buyer_maker
id                                                                           
1987119092  BTCUSDT  43155.0  0.001      43.15  1646092800000            True
1987119093  BTCUSDT  43150.8  0.001      43.15  1646092800027            True
1987119094  BTCUSDT  43151.4  0.019     819.87  1646092800089            True
1987119095  BTCUSDT  43151.4  0.031    1337.69  1646092800089            True
1987119096  BTCUSDT  43151.4  0.001      43.15  1646092800089            True
...             ...      ...    ...        ...            ...             ...
1999773848  BTCUSDT  424

### Sample function call for most recent n days


In [5]:
result=data_downloader(n_days=5)

print(result)

Data Downloading for :  ['2022-04-11', '2022-04-12', '2022-04-13', '2022-04-14', '2022-04-15', '2022-04-16']
https://data.binance.vision/data/futures/um/daily/trades/BTCUSDT/BTCUSDT-trades-2022-04-11.zip
https://data.binance.vision/data/futures/um/daily/trades/BTCUSDT/BTCUSDT-trades-2022-04-12.zip
https://data.binance.vision/data/futures/um/daily/trades/BTCUSDT/BTCUSDT-trades-2022-04-13.zip
https://data.binance.vision/data/futures/um/daily/trades/BTCUSDT/BTCUSDT-trades-2022-04-14.zip
https://data.binance.vision/data/futures/um/daily/trades/BTCUSDT/BTCUSDT-trades-2022-04-15.zip
https://data.binance.vision/data/futures/um/daily/trades/BTCUSDT/BTCUSDT-trades-2022-04-16.zip
             symbol    price    qty  quote_qty           time  is_buyer_maker
id                                                                           
2113048317  BTCUSDT  42140.3  0.010     421.40  1649635200078            True
2113048318  BTCUSDT  42140.4  0.004     168.56  1649635200094           False
211304831